# Treino/Teste modelo

In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import regex as re
import string

## 1. Importando dados da preparação e stopwords

In [2]:
PARQUET_MPO_PREPARADOS = '../../Data/Processed/mpo_preparados.parquet'

df_mpo = pd.read_parquet(PARQUET_MPO_PREPARADOS)

In [3]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marcelo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_mpo.head(5)

,ID,Level,MpoIdentificador,MpoAreaEletrica,MpoCentro,MpoLigadoDesligado,Texto,Título
0,161279,1,IO-PM.6MD,6MD,CNOS,Não definido,Estabelecer os procedimentos a serem seguido...,OBJETIVO
1,161280,2,IO-PM.6MD,6MD,CNOS,Não definido,2.1. Os procedimentos contidos ne...,CONSIDERAÇÕES GERAIS
2,161281,3,IO-PM.6MD,6MD,CNOS,Não definido,1. OBJETIVO 2. ............................,PREPARAÇÃO PARA MANOBRAS EM LINHAS DE TRANSMISSÃO
3,161282,3.1,IO-PM.6MD,6MD,CNOS,Não definido,1. OBJETIVO 2. ............................,LT 230 kV Coletora Porto Velho / Porto Velho C...
4,161283,3.1.1,IO-PM.6MD,6MD,CNOS,Desligar,3.1.1.1. Desenergização da LT 230 ...,Desenergização da LT 230 kV Coletora Porto Vel...


## 2. Vetorizando com tf-idf

In [9]:
#vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords) # , ngram_range = (1, 2)
#X = vectorizer_tfidf.fit_transform(df_mpo['Texto_Limpo'])
vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords, min_df = 0.001, max_df = 0.99) # , ngram_range = (1, 2)
X = vectorizer_tfidf.fit_transform(df_mpo[(df_mpo['MpoLigadoDesligado']=='Desligar') | (df_mpo['MpoLigadoDesligado']=='Ligar')]['Texto'])
vocab = np.array(vectorizer_tfidf.get_feature_names())

In [10]:
#1st run - <5456x8001 sparse matrix of type '<class 'numpy.float64'>'
#	with 306816 stored elements in Compressed Sparse Row format>

#2nd run - 0.05 - 0.95 - <5456x208 sparse matrix of type '<class 'numpy.float64'>'
#	with 170520 stored elements in Compressed Sparse Row format>


#3rd run - 0.01 - 0.99 - <5456x921 sparse matrix of type '<class 'numpy.float64'>'
#	with 263489 stored elements in Compressed Sparse Row format>

X

<3491x2564 sparse matrix of type '<class 'numpy.float64'>'
	with 151516 stored elements in Compressed Sparse Row format>

In [14]:
vocab[2000:2030]

array(['proceder', 'procedimento', 'procedimentos', 'procurar',
       'programa', 'programadas', 'programado', 'programados', 'proibida',
       'proibitivo', 'promissão', 'proprietária', 'proprietário',
       'proteção', 'proveniente', 'providenciar', 'providencie',
       'providências', 'provoca', 'provocar', 'provoque', 'prs', 'pré',
       'prévio', 'próxima', 'próximo', 'pss', 'pte', 'pólo', 'q1'],
      dtype='<U19')

## 4. Modelagem de tópicos com NMF

In [15]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [25]:
from sklearn.decomposition import NMF

#d = X.shape[0]
num_top_words=6

model_nmf = NMF(n_components=100, init='random', random_state=0)

W1 = model_nmf.fit_transform(X)
H1 = model_nmf.components_


print(show_topics(H1))

['lt energização desta sentido tensão partir', 'operador fechamento autonomia oi situações demais', '69 transformador kv 13 230 mva', 'específica lado cemig energização instalação io', '500 kv 550 300 terminal reator', 'iguaçu nova angra taubaté grajaú faixas', 'c2 c1 lt terminal escolhido c3', 'biguaçu palhoça eletrosul florianópolis gaspar ocorrer', '525 itá disjuntores eletrosul kv areia', 'autotransformador lado 13 230 kv ltc', '138 kv lado transformador mva defasador', 'curitiba leste cidade uberaba get copel', 'iniciando instruções instalações envolvidas energização inicia', 'ute viana goiânia suzano araucária coe', 'dutra presidente imperatriz açailândia eletronorte oeorm', 'campo comprido get mourão copel umbará', 'cron ne chesf banabuiú milagres picos', 'ii c1 luís miranda sobral kv', 'nova mutum ponte brasnorte sorriso avanhandava', 'tf 13 transformador lado 230 disjuntor', '345 kv 362 itutinga pimenta viana', 'neves paulino elevadora sete lagoas steag', '01 02 03 nco eletron

In [20]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=8, init='k-means++', max_iter=100, n_init=10, verbose=True)
km.fit_transform(X)

print(show_topics(km.cluster_centers_))

Initialization complete
Iteration 0, inertia 5584.360544613293
Iteration 1, inertia 3035.6876248200574
Iteration 2, inertia 2966.3641811478346
Iteration 3, inertia 2950.205250458826
Iteration 4, inertia 2941.2366988166937
Iteration 5, inertia 2935.3261737498083
Iteration 6, inertia 2932.77777637949
Iteration 7, inertia 2930.2106601774126
Iteration 8, inertia 2929.638998888717
Iteration 9, inertia 2929.462580552924
Iteration 10, inertia 2929.2806406647596
Iteration 11, inertia 2929.0613703714907
Iteration 12, inertia 2928.802314267806
Iteration 13, inertia 2928.602111910723
Iteration 14, inertia 2928.502769117967
Iteration 15, inertia 2928.444835423924
Iteration 16, inertia 2928.40584986647
Iteration 17, inertia 2928.385738355421
Iteration 18, inertia 2928.373476950292
Iteration 19, inertia 2928.3669790698555
Iteration 20, inertia 2928.3579997797315
Iteration 21, inertia 2928.3526723906966
Iteration 22, inertia 2928.346222949199
Iteration 23, inertia 2928.3416766356536
Iteration 24, ine

In [18]:
from collections import Counter
c = Counter(km.labels_)
c

Counter({4: 1600, 0: 781, 2: 333, 3: 676, 1: 101})

In [21]:
y = km.labels_
y

array([5, 5, 5, ..., 7, 7, 7], dtype=int32)

In [23]:
cc = km.cluster_centers_
cc

array([[0.        , 0.00204421, 0.00671521, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00128445, 0.00532407, ..., 0.00254439, 0.00033238,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00200538, 0.0110459 , 0.        , ..., 0.000561  , 0.00073681,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00045494, 0.0022397 , 0.        , ..., 0.00062491, 0.00033502,
        0.00023623]])